In [ ]:
import subprocess

def get_gpu_usage():
    try:
        result = subprocess.check_output(
            ['nvidia-smi', '--query-gpu=utilization.gpu', '--format=csv,noheader,nounits'],
            encoding='utf-8'
        )
        usage = result.strip().split('\n')
        usage = [int(u) for u in usage]
        return usage  # 예: [23, 5] → GPU0: 23%, GPU1: 5%
    except Exception as e:
        print("GPU 사용량 확인 실패:", e)
        return []


In [ ]:
usage = get_gpu_usage()
for i, u in enumerate(usage):
    print(f"GPU {i}: {u}% 사용 중")


In [ ]:
from PyQt5.QtWidgets import QApplication, QWidget, QLabel, QVBoxLayout
import sys

class Example(QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        label = QLabel("배경색이 변경된 라벨입니다.")
        # label.setStyleSheet("background-color: lightblue;") # 간단한 색상 이름 사용
        label.setStyleSheet("background-color: #ADD8E6;") # 16진수 색상 코드 사용

        button = QPushButton("배경색이 변경된 버튼입니다.")
        button.setStyleSheet("background-color: yellow;")

        central_widget = QWidget()
        central_widget.setStyleSheet("background-color: lightgray;") # 위젯 자체의 배경색 변경

        layout = QVBoxLayout(central_widget)
        layout.addWidget(label)
        layout.addWidget(button)

        self.setCentralWidget(central_widget) # QMainWindow의 경우
        if not isinstance(self, QMainWindow): # 일반 QWidget의 경우
            self.setLayout(layout)

        self.setGeometry(300, 300, 300, 200)
        self.setWindowTitle('Change Background Color')
        self.show()

if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = QMainWindow() # 또는 QWidget()
    example = Example()
    window.setCentralWidget(example) # QMainWindow의 경우
    if isinstance(window, QWidget): # 일반 QWidget의 경우
        window = example
    sys.exit(app.exec_())

In [ ]:
from PyQt5.QtWidgets import QWidget
from PyQt5.QtGui import QPainter, QPen, QPolygonF
from PyQt5.QtCore import Qt, QPointF, pyqtSignal
import datetime

class ROIEditor(QWidget):
    """
    ROI 영역을 그리고 확정하는 기능
    """
    roi_defined = pyqtSignal(list, int)  # 추가된 부분: 카메라 ID를 포함한 시그널

    def __init__(self, video_widget, cam_id):
        super().__init__(video_widget)
        self.video_widget = video_widget
        self.cam_id = cam_id  # 카메라 ID 저장
        self.points = []
        self.finished = False
        self.temp_point = None
        self.now = datetime.datetime.now()

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            # 좌클릭으로 점을 추가
            pos = event.pos()
            self.points.append(QPointF(pos))
            self.update()  # 화면 업데이트

        elif event.button() == Qt.RightButton and len(self.points) >= 3:
            # 우클릭 시 폴리곤을 확정
            self.finished = True
            self.temp_point = None
            polygon_coords = [(pt.x(), pt.y()) for pt in self.points]
            self.roi_defined.emit(polygon_coords, self.cam_id)  # 카메라 ID 포함하여 시그널 전송
            self.update()

    def mouseMoveEvent(self, event):
        if not self.finished:
            # 마우스를 이동하면서 마지막 점과 현재 점을 빨간색 선으로 그리기
            self.temp_point = event.pos()
            self.update()

    def paintEvent(self, event):
        print(__class__.__name__)
        print(f"{self.now.second}[DEBUG] paintEvent called for cam_id: {self.cam_id}, points: {self.points}" )
        painter = QPainter(self)

        # 점 찍기 (파란색) - 항상 그리기
        dot_pen = QPen(Qt.blue, 5)
        painter.setPen(dot_pen)
        for pt in self.points:
            painter.drawPoint(pt)

        # 선 그리기 (빨간색 또는 파란색)
        line_pen = QPen(Qt.red, 2)
        painter.setPen(line_pen)

        if self.points:
            polygon = QPolygonF(self.points)
            if self.finished:
                # 다각형이 완성되었으면 첫 점과 마지막 점을 연결하여 파란색으로 그린다.
                blue_pen = QPen(Qt.blue, 2)
                painter.setPen(blue_pen)
                polygon.append(self.points[0])
                painter.drawPolygon(polygon)
            else:
                # 다각형을 그리기 전에 폴리라인으로 점을 연결하고, 마우스를 이동하면서 그려지는 빨간색 선
                painter.drawPolyline(polygon)
                if self.temp_point:
                    painter.drawLine(self.points[-1], self.temp_point)

    def reset(self):
        # ROI 설정 초기화
        self.points.clear()
        self.finished = False
        self.temp_point = None
        self.update()

    def get_polygon(self):
        """[(x1, y1), (x2, y2), ...] 형식으로 반환"""
        return [(pt.x(), pt.y()) for pt in self.points] if self.finished else None

In [ ]:
from PyQt5.QtWidgets import QApplication, QWidget, QPushButton
from PyQt5.QtGui import QColor
from PyQt5.QtCore import Qt

class App(QWidget):
    def __init__(self):
        super().__init__()

        self.setWindowTitle('MN Vision 스타일')
        self.setGeometry(100, 100, 800, 600)

        # 배경색 설정
        self.setStyleSheet("background-color: #1A1A1A;")

        # 버튼 생성
        button = QPushButton('클릭하세요', self)
        button.setGeometry(300, 250, 200, 50)
        button.setStyleSheet("""
            background-color: #FF6600;
            color: #FFFFFF;
            border: none;
            border-radius: 10px;
        """)
        button.setCursor(Qt.PointingHandCursor)

        # 버튼 호버 효과
        button.setStyleSheet("""
            QPushButton {
                background-color: #FF6600;
                color: #FFFFFF;
                border: none;
                border-radius: 10px;
            }
            QPushButton:hover {
                background-color: #00B0FF;
            }
        """)

if __name__ == '__main__':
    app = QApplication([])
    window = App()
    window.show()
    app.exec_()


In [ ]:

# class VideoThread(QThread):
#     frame_ready = pyqtSignal(np.ndarray)
#     event_triggered = pyqtSignal(float, str)
#     mute_triggered = pyqtSignal(str)


#     def __init__(self, video_path, detector, postprocessor, video_buffer):
#         super().__init__()
#         self.cap = cv2.VideoCapture(video_path)
#         self.detector = detector
#         self.postprocessor = postprocessor
#         self.video_buffer = video_buffer
#         self.roi = None
#         self.running = True
#         self.frame_count = 0
        
#     # def run(self):
#     #     while self.running:
#     #         ret, frame = self.cap.read()
#     #         if not ret:
#     #             break
#     #         self.frame_count += 1
#     #         self.video_buffer.add_frame(frame.copy())
#     #         if self.frame_count % 3 != 0:
#     #             continue

#     #         results = self.postprocessor.filter_results(self.detector.detect_objects(frame))

#     #         # ROI 시각화 (파란색 선)
#     #         # if self.roi is not None:
#     #         #     cv2.polylines(frame, [self.roi], isClosed=True, color=(255, 0, 0), thickness=2)

#     #         # 객체 박스 시각화 (초록색 박스 + 라벨)
#     #         for (x1, y1, x2, y2), conf, class_name in results:
#     #             label = f"{class_name} {conf:.2f}"
#     #             cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
#     #             cv2.putText(frame, label, (int(x1), int(y1) - 10),
#     #                         cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

#     #             # ROI 내 이벤트 트리거 체크
#     #             if self.roi is not None and cv2.pointPolygonTest(self.roi, (int(x1), int(y1)), False) >= 0:
#     #                 self.event_triggered.emit(time.time(), label)

#     #         self.frame_ready.emit(frame)
#     def run(self):
#         while self.running:
#             ret, frame = self.cap.read()
#             if not ret:
#                 break
#             self.frame_count += 1
#             self.video_buffer.add_frame(frame.copy())
#             if self.frame_count % 3 != 0:
#                 continue

#             results = self.postprocessor.filter_results(self.detector.detect_objects(frame))

#             for det in results:
#                 x1, y1, x2, y2 = det['box']
#                 conf = det['conf']
#                 class_name = det['class_name']
#                 label = f"{class_name} {conf:.2f}"

#                 # # 객체 박스 시각화 (초록색 박스 + 라벨)
#                 # cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
#                 # cv2.putText(frame, label, (int(x1), int(y1) - 10),
#                 #             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

#                 ## 'forklift_left' 일 경우 뮤트 트리거.
#                 ##  연결부 아직 없음. 스피커로 연결할듯?
#                 if class_name == 'forklift_left':
#                     self.mute_triggered.emit(time.time(), label)
                
#                 # 세그멘테이션 폴리곤이 있다면 시각화
#                 if det.get('polygons'):
#                     if class_name == 'person':
#                         cv2.putText(frame, label, (int(x1), int(y1) - 10),
#                                 cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
#                         for poly in det['polygons']:
#                             poly_np = np.array(poly, dtype=np.int32)
#                             cv2.polylines(frame, [poly_np], isClosed=True, color=(0, 255, 0), thickness=2)
#                     else:
#                         cv2.putText(frame, label, (int(x1), int(y1) - 10),
#                                 cv2.FONT_HERSHEY_SIMPLEX, 0.5, (205, 205, 0), 1)
#                         for poly in det['polygons']:
#                             poly_np = np.array(poly, dtype=np.int32)
#                             cv2.polylines(frame, [poly_np], isClosed=True, color=(205, 205, 0), thickness=2)

#                 # ROI 내 이벤트 트리거 체크
#                 if self.roi is not None and cv2.pointPolygonTest(self.roi, (int(x1), int(y1)), False) >= 0:
#                     self.event_triggered.emit(time.time(), label)

#             self.frame_ready.emit(frame)

#     def calculate_iou(poly1, poly2):
#         poly1 = Polygon(poly1)
#         poly2 = Polygon(poly2)
#         if not poly1.is_valid or not poly2.is_valid:
#             return 0
#         intersection_area = poly1.intersection(poly2).area
#         union_area = poly1.union(poly2).area
#         if union_area == 0:
#             return 0
#         return intersection_area / union_area

#     def check_person_forklift_overlap(detections, iou_threshold=0.2):
#         """
#         person 객체와 forklift-* 객체의 polygon IoU를 계산하고 일정 임계값을 넘으면 경고 이벤트 발생
#         :param detections: [{'class_name': str, 'polygons': list[tuple], ...}, ...]
#         """
#         person_polys = [Polygon(d['polygons']) for d in detections if d['class_name'] == 'person']
#         forklift_polys = [Polygon(d['polygons']) for d in detections if d['class_name'].startswith('forklift')]

#         for person_poly in person_polys:
#             for forklift_poly in forklift_polys:
#                 if not person_poly.is_valid or not forklift_poly.is_valid:
#                     continue  # 잘못된 polygon 예외 처리

#                 inter_area = person_poly.intersection(forklift_poly).area
#                 union_area = person_poly.union(forklift_poly).area
#                 if union_area == 0:
#                     continue  # divide-by-zero 방지

#                 iou = inter_area / union_area
#                 if iou >= iou_threshold:
#                     print(f"⚠️ 위험 감지: person과 forklift 겹침 (IoU={iou:.2f})")
#                     return True  # 또는 이벤트 emit 호출

#         return False

    

#     def set_roi(self, roi_points):
#         self.roi = np.array(roi_points, dtype=np.int32)
    
#     def stop(self):
#         self.running = False
#         self.cap.release()

In [ ]:
from PyQt5.QtCore import QObject, pyqtSignal, pyqtSlot
from PyQt5.QtMultimedia import QSound
import time
import threading
import os
'./resources/etc/forklift_away.wav'
sound_path = './resources/etc/forklift_away.wav'
sound = QSound('./resources/etc/forklift_away.wav')
current_sound = sound
print("[🔈 재생 시작]")
current_sound.play()
# time.sleep(3)  # 예시: 3초 대기
print("[🧵 스레드 종료]")

[🧵 스레드 시작]
[🔈 재생 시작]
[🧵 스레드 종료]


In [ ]:
import sys
from PyQt5.QtWidgets import QApplication
from PyQt5.QtMultimedia import QSound
import os
app = QApplication(sys.argv)
print(os.path.abspath('./resources/etc/forklift_away.wav'))
print(os.path.exists('./resources/etc/forklift_away.wav'))


sound = QSound('./resources/etc/forklift_away.wav')
print("[🔈 재생 시작]")
sound.play()

# 이벤트 루프 실행 (소리가 재생되는 동안 유지)
# QSound는 소리가 끝날 때까지 이벤트 루프가 살아 있어야 소리가 재생됩니다.
app.exec_()


c:\Users\kdt\OneDrive\바탕 화면\PROJECT_MNV\App\resources\etc\forklift_away.wav
True
[🔈 재생 시작]


In [8]:
import os, subprocess

def openFolder(folder_path='./resources/logs'):
    """ 
    폴더 열기 메서드
    """
    # folder_path = os.path.join(folder_path, str(1))  # 여기에 열고 싶은 폴더 경로 입력
    # folder_path = folder_path+'/1'  # 여기에 열고 싶은 폴더 경로 입력
    folder_path = 'resources/logs/1'
    if os.path.exists(folder_path):
        subprocess.Popen(f'explorer "{folder_path}"')

        print(f"{folder_path}")
    else:
        print(f"경로가 존재하지 않습니다. {folder_path}")
# C:\Users\kdt\OneDrive\바탕 화면\PROJECT_MNV\App\resources\logs\1        
openFolder()

resources/logs/1
